#Features from bureau_balance dataset

Этот блокнот создает признаки из набора данных bureau_balance. Набор данных bureau_balance содержит информацию о ежемесячном остатке прошлых кредитов (активных или закрытых), предоставленных другими финансовыми учреждениями, которые были переданы в Бюро кредитных историй. Application связан с набором данных bureau через признак SK_ID_BUREAU.

In [ ]:
import numpy as np
import pandas as pd

# -----------------------------------------------------
from google.colab import drive

# -----------------------------------------------------
import zipfile
import time
import sys
import os
import gc

In [ ]:
# Сброс ограничений на число столбцов
pd.set_option("display.max_columns", None)

# Сброс ограничений на число строк до 400
pd.set_option("display.max_rows", 400)

In [ ]:
# Подключение Google Drive к текущей среде выполнения

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Функция распаковки содержимого архива в указанную директорию

def extract_zip_file(zip_path, extract_path):
    """
    Распаковка содержимого архива в указанную директорию

    :param zip_path: Путь к архиву
    :param extract_path: Путь к директории, куда нужно распаковать содержимое архива
    """
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_path)

In [ ]:
# Распаковка содержимого архива home-credit-default-risk в текущую директорию

zip_file = "/content/drive/My Drive/home-credit-default-risk.zip"
extract_path = "/content/"

extract_zip_file(zip_file, extract_path)

In [ ]:
# Распаковка содержимого архива 2 - Bureau features в текущую директорию

zip_file = "/content/drive/My Drive/2 - Bureau features.zip"
extract_path = "/content/"

extract_zip_file(zip_file, extract_path)

In [ ]:
# Чтение данных

application = pd.read_csv("/content/2 - Bureau features/application.csv")

bureau = pd.read_csv("/content/home-credit-default-risk/bureau.csv")
bureau_balance = pd.read_csv("/content/home-credit-default-risk/bureau_balance.csv")
homecredit_columns_description = pd.read_csv("/content/2 - Bureau features/homecredit_columns_description.csv", encoding="ISO-8859-1")

In [ ]:
# Проверка

application.shape

(356255, 169)

In [ ]:
# Описание столбцов в bureau_balance
description_application = homecredit_columns_description[homecredit_columns_description["Table"] == "bureau_balance.csv"]
for index, row in description_application.iterrows():
    print(row["Row"], row["Description"])

# Изменение данных STATUS для преобразования в целочисленный тип
bureau_balance.loc[bureau_balance["STATUS"] == "C", "STATUS"] = -2
bureau_balance.loc[bureau_balance["STATUS"] == "X", "STATUS"] = -1
bureau_balance["STATUS"] = bureau_balance["STATUS"].astype(int)

bureau_balance.head(20)

SK_BUREAU_ID Recoded ID of Credit Bureau credit (unique coding for each application) - use this to join to CREDIT_BUREAU table 
MONTHS_BALANCE Month of balance relative to application date (-1 means the freshest balance date)
STATUS Status of Credit Bureau loan during the month (active, closed, DPD0-30,ÃÂ [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,ÃÂ 5 means DPD 120+ or sold or written off ] )


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,-2
1,5715448,-1,-2
2,5715448,-2,-2
3,5715448,-3,-2
4,5715448,-4,-2
5,5715448,-5,-2
6,5715448,-6,-2
7,5715448,-7,-2
8,5715448,-8,-2
9,5715448,-9,0


In [ ]:
# Объединение bureau_balance и bureau для получения связи с application (SK_ID_CURR)

bureau_balance_id = bureau_balance.merge(bureau[["SK_ID_CURR", "SK_ID_BUREAU"]], on ="SK_ID_BUREAU", how = "left")
bureau_balance_id = bureau_balance_id.dropna(subset=["SK_ID_CURR"])
bureau_balance_id["SK_ID_CURR"] = bureau_balance_id["SK_ID_CURR"].astype(int)

In [ ]:
# Извлечение признаков из bureau_balance

# Создание пустых списков для каждого нового признака
nb_months_list = []
nb_late_payments_list = []
nb_late_payments_3m_list = [] 
nb_late_payments_12m_list = []
nb_late_payments_1_list = []
nb_late_payments_2_list = [] 
nb_late_payments_3_list = []
nb_late_payments_4_list = []
nb_late_payments_5_list = []
fr_late_payments = []

counter = 0
lenght = len(application)
for index, row in application.iterrows():
    counter += 1
    sys.stdout.write('\r'+"Progress:"+str(round(counter/lenght*100,1))+"%")
    df = bureau_balance_id[bureau_balance_id["SK_ID_CURR"] == row["SK_ID_CURR"]]
    if len(df) > 0:
        # Общее количество месяцев
        nb_months = len(df)             
        # Общее количество просроченных платежей
        nb_late_payments = len(df[df["STATUS"] > 0])
        # Количество просроченных платежей за последние 3 месяца
        tmp = df[df["MONTHS_BALANCE"] >= -3]
        nb_late_payments_3m = len(tmp[tmp["STATUS"] > 0])
        # Количество просроченных платежей за последние 12 месяцев
        tmp = df[df["MONTHS_BALANCE"] >= -12]
        nb_late_payments_12m = len(tmp[tmp["STATUS"] > 0])
        # Количество просроченных платежей с пометкой "1"
        nb_late_payments_1 = len(df[df["STATUS"] == 1])
        # Количество просроченных платежей с пометкой "2"
        nb_late_payments_2 = len(df[df["STATUS"] == 2]) 
        # Количество просроченных платежей с пометкой "3"
        nb_late_payments_3 = len(df[df["STATUS"] == 3])
        # Количество просроченных платежей с пометкой "4"
        nb_late_payments_4 = len(df[df["STATUS"] == 4]) 
        # Количество просроченных платежей с пометкой "5"
        nb_late_payments_5 = len(df[df["STATUS"] == 5])
        nb_months_list.append(nb_months)
        nb_late_payments_list.append(nb_late_payments)
        nb_late_payments_3m_list.append(nb_late_payments_3m)
        nb_late_payments_12m_list.append(nb_late_payments_12m)
        nb_late_payments_1_list.append(nb_late_payments_1)
        nb_late_payments_2_list.append(nb_late_payments_2)
        nb_late_payments_3_list.append(nb_late_payments_3)
        nb_late_payments_4_list.append(nb_late_payments_4)
        nb_late_payments_5_list.append(nb_late_payments_5)
        # Доля месяцев с просроченными платежами в bureau balance
        if nb_months > 0:
            fr_late_payments.append(nb_late_payments/nb_months)
        else:
            fr_late_payments.append(np.NaN)
    else:
        nb_months_list.append(0)
        nb_late_payments_list.append(0)
        nb_late_payments_3m_list.append(0)
        nb_late_payments_12m_list.append(0)
        nb_late_payments_1_list.append(0)
        nb_late_payments_2_list.append(0)
        nb_late_payments_3_list.append(0)
        nb_late_payments_4_list.append(0)
        nb_late_payments_5_list.append(0)
        fr_late_payments.append(np.NaN)

Progress:100.0%

In [ ]:
# Функция для отображения описания выбранного признака

def add_description(table, row, description):
    list_description = [np.NaN, table, row, description, np.NaN]
    homecredit_columns_description.loc[len(homecredit_columns_description)] = list_description
    return homecredit_columns_description

In [ ]:
# Создание новых признаков из bureau_balance

application["nb_months_list"] = nb_months_list
add_description("bureau_balance.csv", "nb_months_list", "Total number of months in bureau balance")

application["nb_late_payments_list"] = nb_late_payments_list
add_description("bureau_balance.csv", "nb_late_payments_list", "Total number of late payment months in bureau balance")

application["nb_late_payments_3m_list"] = nb_late_payments_3m_list
add_description("bureau_balance.csv", "nb_late_payments_3m_list", "Number of late payments in the last 3 months in bureau balance")

application["nb_late_payments_12m_list"] = nb_late_payments_12m_list
add_description("bureau_balance.csv", "nb_late_payments_12m_list", "Number of late payments in the last 12 months in bureau balance")

application["nb_late_payments_1_list"] = nb_late_payments_1_list
add_description("bureau_balance.csv", "nb_late_payments_1_list", "Number of category 1 late payments months in bureau balance")

application["nb_late_payments_2_list"] = nb_late_payments_2_list
add_description("bureau_balance.csv", "nb_late_payments_2_list", "Number of category 2 late payments months in bureau balance")

application["nb_late_payments_3_list"] = nb_late_payments_3_list
add_description("bureau_balance.csv", "nb_late_payments_3_list", "Number of category 3 late payments months in bureau balance")

application["nb_late_payments_4_list"] = nb_late_payments_4_list
add_description("bureau_balance.csv", "nb_late_payments_4_list", "Number of category 4 late payments months in bureau balance")

application["nb_late_payments_5_list"] = nb_late_payments_5_list
add_description("bureau_balance.csv", "nb_late_payments_5_list", "Number of category 5 late payments months in bureau balance")

application["fr_late_payments"] = fr_late_payments
add_description("bureau_balance.csv", "fr_late_payments", "Fraction of months with late payments in bureau balance");

In [ ]:
# Проверка

application.shape

(356255, 179)

In [ ]:
# Сохранение application

application.to_csv("application.csv", index=False)

In [ ]:
# Сохранение homecredit_columns_description

homecredit_columns_description.to_csv("homecredit_columns_description.csv", index=False)